In [34]:
!pip install praw

In [35]:
!pip install nltk

In [36]:
!pip install openai

In [37]:
pip install ipywidgets


Note: you may need to restart the kernel to use updated packages.


In [38]:
import praw
import pandas as pd
import re
import datetime
from tqdm import tqdm
import os
from openai import OpenAI
import nltk

In [39]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dalia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

scrape reddit data and build from it 'reddit_data_IsraelPalestine.csv' file

In [40]:
def remove_text_between_brackets(text):
    pattern = r'\[.*?\]'
    result = re.sub(pattern, '', text)
    return result

def num_words(text):
    return len(text.split())


def convert_utc_to_datetime(utc_time):
    return datetime.datetime.utcfromtimestamp(utc_time).strftime('%d/%m/%y %H:%M:%S')

def remove_text_after_x(text, x):
    pattern = re.escape(x) + r'.*'
    result = re.sub(pattern, '', text, flags=re.IGNORECASE)
    return result

def extract_data_from_comment(comment, prefix=None):
    data = dict(comment_id=comment.id, comment_body=remove_text_after_x(comment.body, 'edit:'),
                comment_date_time=convert_utc_to_datetime(comment.created), comment_score=comment.score)
    if prefix is not None:
        data = {f"{prefix}_{k}": v for k, v in data.items()}
    return data

def extract_data_from_post(post):
    post_data = dict(post_id=post.id, title=remove_text_between_brackets(post.title),
                     body=remove_text_after_x(post.selftext, 'edit:'), date_time=convert_utc_to_datetime(post.created),
                     url=post.url, score=post.score, num_comments=post.num_comments, post_len = num_words(remove_text_after_x(post.selftext, 'edit:')))
    return post_data

In [41]:
subrredit_str = 'IsraelPalestine'
queries = ['Israel', 'Israeli', 'Palestine', 'Palestinian', 'Palestinians', 'Hamas', 'Oct 7', 'Oct 2023', 'Gaza']
time_filters = ['all'] # ['all', 'year', 'month', 'week']
sorts = ['top'] # ['top', 'relevance', 'new']

In [42]:
CLIENT_ID = 'e4m-_9FCjrOOKVVdo1SIBw'
CLIENT_SECRET = 'LP_WMFy4Y8CanKvn1M5bppXayfwONA'
USER_AGENT = 'Nevonot_project'  # just give it a name
reddit = praw.Reddit(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, user_agent=USER_AGENT)
subrredit = reddit.subreddit(subrredit_str)

In [43]:
data = []
for q in queries:
    for tf in time_filters:
        for so in sorts:
            print(q, tf, so)
            posts = list(subrredit.search(query=q, sort=so, time_filter=tf, limit = 5000))
            print(f"num of sample under {q} symptom: ", len(posts))
            for post in tqdm(posts):
                try:
                    post_data = extract_data_from_post(post)
                    post_data.update(dict(query=q, sort=so, time_filter=tf))
                    data.append(post_data)
                except Exception as e:
                    pass
data = pd.DataFrame(data)
data = data[data['score'] >= 0]
print(len(data))
unique_data = data.drop_duplicates(subset='post_id', keep='first')
print(len(unique_data))
unique_data.to_csv("reddit_data_IsraelPalestine.csv", index=False, encoding='utf-8-sig')
unique_data

Israel all top
num of sample under Israel symptom:  230


100%|██████████████████████████████████████████████████████████████████████████████| 230/230 [00:00<00:00, 7604.25it/s]

Israeli all top


num of sample under Israeli symptom:  226


100%|██████████████████████████████████████████████████████████████████████████████| 226/226 [00:00<00:00, 9761.73it/s]

...

Gaza all top


num of sample under Gaza symptom:  234


100%|██████████████████████████████████████████████████████████████████████████████| 234/234 [00:00<00:00, 7123.64it/s]

1952
752


,post_id,title,body,date_time,url,score,num_comments,post_len,query,sort,time_filter
0,172w9gt,I have changed my mind about the Israel-Palest...,For the past decade or so I have typically sup...,08/10/23 10:51:36,https://www.reddit.com/r/IsraelPalestine/comme...,4893,2810,175,Israel,top,all
1,1736ce5,I am done with the the deafening silence from ...,I think it's time to discuss what will happen ...,08/10/23 18:34:34,https://www.reddit.com/r/IsraelPalestine/comme...,1594,1191,88,Israel,top,all
2,nb3trt,Both sides are being mislead of the full truth,So this is my compilation of how this whole th...,13/05/21 00:27:49,https://www.reddit.com/r/IsraelPalestine/comme...,1326,645,952,Israel,top,all
3,176ttyf,Lets set things straight,"Hey reddit , My name is Ofek.\nI was an israel...",13/10/23 08:19:55,https://www.reddit.com/r/IsraelPalestine/comme...,1058,1916,441,Israel,top,all
4,nem47n,You can be anti-Hamas but pro-Palestine,I believe that Hamas is a very dangerous terro...,17/05/21 17:20:14,https://www.reddit.com/r/IsraelPalestine/comme...,941,592,73,Israel,top,all
...,...,...,...,...,...,...,...,...,...,...,...
1945,18n31wr,"Why Palestinian cities are called ""refugee cam...",Please take a few minutes and read it until t...,20/12/23 19:32:10,https://www.reddit.com/r/IsraelPalestine/comme...,114,390,598,Gaza,top,all
1947,1bq3j7b,My thoughts about the war,"\nHello everybody, I am an Israeli soldier, c...",28/03/24 18:57:05,https://www.reddit.com/r/IsraelPalestine/comme...,114,173,744,Gaza,top,all
1948,ods0eq,Arab crimes against Arabs are worse than israel.,Recently a Lebanese journalist criticized the ...,04/07/21 20:09:08,https://www.reddit.com/r/IsraelPalestine/comme...,111,122,103,Gaza,top,all
1949,1an112x,"HAMAS EXPOSED, the body is not greater than th...","\nYahya Sinwar, the Hamas leader, climbed the ...",09/02/24 22:49:24,https://www.reddit.com/r/IsraelPalestine/comme...,112,107,593,Gaza,top,all


gpt api pipeline

In [44]:
## Don't share it!!!!!.
os.environ["OPENAI_API_KEY"] = "YourKeyHere"
client = OpenAI(
    api_key=os.environ['OPENAI_API_KEY'],
)

In [45]:
def is_sentence_complete(text):
    # Use nltk to check if the last sentence is complete
    sentences = nltk.sent_tokenize(text)
    if sentences:
        last_sentence = sentences[-1]
        return last_sentence.endswith('.') or last_sentence.endswith('!') or last_sentence.endswith('?')
    return False

def truncate_text(text):
    while not is_sentence_complete(text):
        # Remove the last word until the sentence is complete
        text = ' '.join(text.split()[:-1])
    return text

In [46]:


def prompt (given_text, mode):
    if mode == "reply":
        system_content = "You are an AI assistant specialized in writing professional pro-Israel posts. Given an anti-Israel post, generate a respectful and factual post that clarifies the situation in Gaza, addresses the complexities of the Israeli-Palestinian conflict, and highlights Israel's efforts towards peace and security for all."
        user1 =  "Given the following anti-Israel post I encountered, 'Today, we stand in solidarity with the people of Palestine, facing ongoing challenges and hardships. As we reflect on the situation, we are drawn to several critical issues that demand our immediate attention and action: The expansion of IDF operations in all areas of Gaza, The continued blockade of Gaza, which affects the daily lives of its residents, The daily hardships experienced by Palestinians living under occupation. We advocate for justice, human rights, and a peaceful resolution that respects the rights and dignity of everyone in the region. It is our firm belief that the safety and freedom of Palestinians are paramount. We call upon the international community to support efforts toward securing a fair and lasting peace. Let's work together for a future where everyone in the region can live in peace and dignity. #SolidarityWithPalestine #JusticeForAll', please generate a respectful and factual pro-israel response post:"
        assistant1 = "Today, I want to address the complexities of the Israeli-Palestinian conflict, especially after coming across several inaccurate posts recently. The IDF's actions in Gaza are defensive measures in response to severe provocations, including incursions into Israeli territory that have led to the killing and abduction of thousands. These actions aim to protect Israeli citizens, with significant efforts made to minimize harm to civilians. Despite these challenges, Israel continues to allow the entry of aid into Gaza, showing its commitment to the welfare of Gaza's residents, even amid conflict. Let's approach this issue with open eyes and minds, seeking truth beyond the distortions. By understanding the facts and working together, we can challenge misinformation and build a future characterized by peace for everyone. #PeaceForAll #DialogueNotDivision"
        prompt = f"Given the following anti-Israel post I encountered, '{given_text}', please generate a respectful and factual pro-israel response post:"

    elif mode == "mystyle":
       system_content = "You are an AI assistant specialized in crafting professional pro-Israel posts that align with a person's style. Given a specific person's previous post, create a new pro-Israel post that accurately reflects on the events of October 7th. This post should address the complexities of the situation while highlighting Israel's efforts towards peace and security for all. The new post must maintain the same tone, style, and respectfulness as the provided post."
       user1 = "Based on the tone and style of my previous post: 'As a vibrant democracy in the Middle East, Israel continues to be a beacon of freedom and advancement. Despite facing unprecedented challenges, it remains committed to its democratic values and the protection of the rights of all its citizens. Israel's contributions to the world are unparalleled, especially in technology and healthcare. Israeli innovations, from drip irrigation to life-saving medical devices, have not only transformed industries but have also improved countless lives globally. It's this spirit of innovation and resilience that inspires and deserves recognition. Here's to Israel's unwavering pursuit of progress and peace. #Israel #Democracy #Innovation #GlobalImpact' , please generate a new pro-Israel post, while maintaining the same level of tone, style, and respectfulness."
       assistant1 = "In the midst of the current conflict, Israel stands resilient, navigating the complexities of warfare with a steadfast commitment to the safety and security of its citizens. Despite challenges and pain, the nation defends its sovereignty against unprecedented threats with unity and determination. The resilience of the Israeli community, rallying together in support of one another, showcases the strength and solidarity defines Israel. This period of conflict highlights the importance of striving for peace and stability in the region, with Israel continuing to seek a path forward that ensures the security and prosperity of all its inhabitants. The nation's enduring hope for a peaceful resolution stands as a testament to its unwavering commitment to the principles of democracy and human rights. As we witness the challenges Israel faces, let us recognize the courage of its people and the nation's relentless pursuit of peace amidst adversity. Here's to Israel's strength, resilience, and unwavering spirit in the face of conflict. #IsraelStrong #Unity #Peace #Resilience"
       prompt = f"Based on the tone and style of my previous post: '{given_text}' , please generate a new pro-Israel post, while maintaining the same level of tone, style, and respectfulness."

    elif mode == "rewrite":
        system_content = "You are an AI assistant specialized in enhancing pro-Israel social media content. Given an original post, your task is to rewrite it to be more fluent and engaging, while ensuring the content remains accurate and suitable for social media. The revised post should capture the essence of the original message but in a way that is more likely to resonate with a wider audience. "
        user1 =  "Given the original post: 'hey guys, just wanted 2 say that Israel's going thru some tough times rn with the current war & all. since oct 7 things have really heated up & it's kinda crazy. Israel's just tryna protect its people & keep everyone safe, but it's like no matter what they do, they get blamed. they're doing some real cool stuff with tech to defend against attacks & still, they make sure to help out with aid & stuff even when things are super tense. it's really important to stick by Israel during these times cuz they're fighting not just for their own safety but for peace & stability in the region. let's show some support for their bravery & all the efforts they're making to keep things from getting worse. peace out.', please rewrite it to be more engaging and suitable for social media, ensuring it remains respectful, factual, and aligns with pro-Israel messaging"
        assistant1 = "Hello everyone, I wanted to take a moment to discuss the challenging situation that Israel is currently facing amidst the ongoing conflict. Since October 7th, the tensions have significantly escalated, presenting a complex scenario. Israel is committed to ensuring the safety and security of its citizens, navigating through these difficult times with a focus on defense and protection. Despite the complexities of the situation, Israel has been proactive in deploying advanced technological solutions to defend against attacks, while also continuing to provide humanitarian aid and support, even under strained circumstances. It's crucial that we stand in solidarity with Israel now more than ever, as the nation strives not only for the safety of its people but also for the peace and stability of the entire region. Let's acknowledge the courage and the extensive efforts Israel is making towards mitigating this crisis. Thank you for your support."
        prompt = f"Given the original post: '{given_text}', please rewrite it to be more engaging and suitable for social media, ensuring it remains respectful, factual, and aligns with pro-Israel messaging"

    elif mode == "completely_new":
        reddit_data = open("reddit_data_IsraelPalestine.csv", "r", encoding='utf-8-sig')
        reddit_data = pd.read_csv(reddit_data)
        reddit_comment = reddit_data.sample(n=1)

        system_content = "You are an AI assistant specialized in creating engaging and professional pro-Israel social media post. Given a Reddit post discussing a specific topic related to Israel, regardless of its original stance, your task is to create a new pro-Israel post. This new post should discuss the topic at hand in a positive light, emphasizing Israel's efforts towards peace and security. Ensure that the new content is factual, respectful, and aligns with pro-Israel messaging, aiming to enlighten and engage a diverse audience."
        user1 = "Given this Reddit post 'I think it's time to discuss what will happen to the people living in Gaza. They are under Hamas rule. Hamas is going use their deaths for further justification for a prolonged conflict in Israel. I don't see a resolution in sight but I do see the hard reality on the ground. Now that this heinous act has occurred and the true face of militant Islam has been shown it is up to the nonviolent majority of muslims to speak out against these disgusting acts of violence.', please create a completely new pro-Israel post. Ensure the post is engaging, respectful, and suitable for a broad social media audience, providing a fresh perspective that aligns with pro-Israel messaging."
        assistant1 = "In light of recent discussions on the challenges faced by the people of Gaza under Hamas' rule, it's important to recognize Israel's unwavering commitment to ensuring the safety and security of all individuals in the region. Israel continues its efforts to provide humanitarian aid, striving to protect innocent lives,  working tirelessly to differentiate between the innocent and those who choose violence. Israel's dedication to fostering a peaceful future is clear, as it works towards solutions that promise security and prosperity for everyone involved, including those in Gaza affected by Hamas' actions. #StandWithIsrael #Peace #BringThemBack"
        prompt = f"Given this Reddit post '{reddit_comment}', please create a completely new pro-Israel post. Ensure the post is engaging, respectful, and suitable for a broad social media audience, providing a fresh perspective that aligns with pro-Israel messaging."

    else:
        print(mode)
        return

    prompt_whole = [{"role": "system", "content": system_content},
                    {"role": "user", "content": user1},
                    {"role": "assistant", "content": assistant1},
                    {"role": "user", "content": prompt}]
    return prompt_whole

In [47]:
temperture_input = 1
max_tokens_input=100

def generate_textual_response (given_text, mode,temperture_input,max_tokens_input, model = "gpt-4"):
    prompt_whole = prompt (given_text, mode)
    try:
        response = client.chat.completions.create(
            model=model,
            messages=prompt_whole,
            temperature=temperture_input,
            max_tokens=max_tokens_input)
    except:
        print(f"Timeout exception occurred. ")
        return

    # Retrieve the text generated by ChatGPT.
    generated_text_initial = response.choices[0].message.content if hasattr(response.choices[0].message,'content') else ''
    generated_text = truncate_text(generated_text_initial)
    return generated_text
    # print(generated_text)

In [48]:
def generate_translation (given_text, from_l, to_l, model = "gpt-3.5-turbo"):

    prompt_whole = [{"role": "system", "content": "You are an AI assistant specialized in providing accurate and contextually appropriate translations. Your task is to translate the given text from the original language into the target language."},
                    {"role": "user", "content": f"Please translate the following text from {from_l} to {to_l}: {given_text}"}]
    try:
        response = client.chat.completions.create(
            model=model,
            messages=prompt_whole,
            temperature=1,
            max_tokens=400)
    except:
        print(f"Timeout exception occurred. ")

    # Retrieve the text generated by ChatGPT.
    generated_text_initial = response.choices[0].message.content if hasattr(response.choices[0].message,'content') else ''
    generated_text = truncate_text(generated_text_initial)
    return generated_text
    # print(generated_text)

In [49]:
import ipywidgets as widgets

In [50]:
from ipywidgets import widgets, Layout, Button
from IPython.display import display, HTML
from functools import wraps
import threading
from threading import Timer

def debounce(wait):
    def decorator(func):
        def debounced(*args, **kwargs):
            def call_it():
                func(*args, **kwargs)
            try:
                debounced.t.cancel()
            except AttributeError:
                pass
            debounced.t = Timer(wait, call_it)
            debounced.t.start()
        return debounced
    return decorator

In [51]:
import base64


# Define the CSS style for the button with the image as background
button_style = f'''
    background-size: cover;
    color: transparent;  /* Hide the text */
    padding: 10px 20px;
    border: none;
    cursor: pointer;
'''



In [52]:
from datetime import datetime

responses_df = pd.DataFrame(columns=['Mode', 'Text', 'Response', 'Date'])


In [53]:
btn_regen = widgets.Button(description='Regenerate')
btn_regen.style.button_color = '#D3D3D3' 
btn_regen.style.font_weight = 'bold'
btn_regen.layout.visibility = 'hidden'  # Initially hidden
regen_output = widgets.Output()

In [54]:
btn_translate = widgets.Button(description='Translate')
btn_translate.style.button_color = '#D3D3D3' 
btn_translate.style.font_weight = 'bold'

btn_translate.layout.visibility = 'hidden'  # Initially hidden
translation_output = widgets.Output()


length_slider = widgets.IntSlider(min=100,
    max=500,value=300,description='')
length_slider.visibility = 'hidden'

creativity_slider = widgets.FloatSlider(min=0,
    max=2,value=1,description='',)
creativity_slider.visibility = 'hidden'

length_slider_labels = widgets.HBox([widgets.Label(value='shorter!', layout=widgets.Layout(width='70px', text_align='right')),
                                     widgets.Label(value='longer!', layout=widgets.Layout(width='70px', text_align='left'))])

creativity_slider_labels = widgets.HBox([widgets.Label(value='less creative!', layout=widgets.Layout(width='90px', text_align='right')),
                                         widgets.Label(value='more creative!', layout=widgets.Layout(width='90px', text_align='left'))])



In [55]:
btn_rep = widgets.Button(description='reply to this text',button_style='success', disabled=True)
btn_rep.style.button_color = 'lightgreen'
btn_rep.style.font_weight = 'bold'

btn_my_style = widgets.Button(description='my style',button_style='info', disabled=True)
btn_my_style.tooltip = "generate a post compatible with the style of the given text"
btn_my_style.style.button_color = 'lightblue'
btn_my_style.style.font_weight = 'bold'


btn_rewrite = widgets.Button(description='rewrite', button_style='warning', disabled=True)
btn_rewrite.style.button_color = 'lightcoral'
btn_rewrite.style.font_weight = 'bold'

btn_new = widgets.Button(description='generate a new post', button_style='danger')
btn_new.style.button_color = 'lightsalmon'
btn_new.style.font_weight = 'bold'

text_input = widgets.Textarea(value="Enter your text here", layout=widgets.Layout(width='50%', height='200px'))

output_widget = widgets.Output()



In [56]:
def on_text_change(change):
    if len(change['new']) > 0:
        btn_rewrite.disabled = False
        btn_my_style.disabled = False
        btn_rep.disabled = False
    else:
        btn_rewrite.disabled = True
        btn_my_style.disabled = True
        btn_rep.disabled = True

In [57]:
from IPython.display import clear_output
translation_text = ""
output_widget = widgets.Output()
translation_output = widgets.Output()
def clear_and_display_output(output):
    with output_widget:
        output_widget.clear_output(wait=True)
        display(HTML(f"<p>{output}</p>"))
    btn_translate.layout.visibility = 'visible'
    btn_regen.layout.visibility = 'visible'
    length_slider.layout.visibility = 'visible'
    creativity_slider.layout.visibility='visible'



In [58]:
def on_translate_click(btn):
    text = responses_df.iloc[-1]["Response"]  # Translate the displayed text
    from_l = "English"
    to_l = "Hebrew" 
#     model = "gpt-3.5-turbo"
    model = "gpt-4"
    translation = generate_translation(text, from_l, to_l, model)
    translation_output.clear_output()
    with translation_output:
        print(translation)
#     btn_translate.layout.visibility = 'hidden'
#     length_slider.visibility = 'hidden'
#     creativity_slider.visibility = 'hidden'
        
btn_translate.on_click(on_translate_click)

In [59]:
def on_regen_click(btn):
    global responses_df
    text = responses_df.iloc[-1]["Text"]  # Translate the displayed text
    mode = responses_df.iloc[-1]["Mode"]
    model = "gpt-4"
 
#     model = "gpt-3.5-turbo"
    output = generate_textual_response(text,mode,temperture_input,max_tokens_input,model)
    date_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")    
    new_row = pd.DataFrame({'Mode': [mode], 'Text': [text], 'Response': [output], 'Date': [date_time]})
    responses_df = pd.concat([responses_df, new_row], ignore_index=True)
    regen_output.clear_output()
    clear_and_display_output(output)

#     btn_translate.layout.visibility = 'hidden'
#     length_slider.visibility = 'hidden'
#     creativity_slider.visibility = 'hidden'
        
btn_regen.on_click(on_regen_click)

In [60]:
def handle_slider_change(change):
    global temperture_input, max_tokens_input
    if change['owner'] == length_slider:
        max_tokens_input = change['new']
    elif change['owner'] == creativity_slider:
        temperture_input = change['new']

length_slider.observe(handle_slider_change, names='value')
creativity_slider.observe(handle_slider_change, names='value')

In [61]:
from IPython.display import HTML
output_display = HTML("")


@debounce(1)
def on_btn_click(btn):
    btn_translate.layout.visibility = 'hidden'# Hide translate button initially
#     for b in buttons:
#         btn.style.border_width = '1px'
#     btn.style.border_width = '2px'
    translation_output.clear_output()
    global responses_df
    text = text_input.value
    x=""
#     model = "gpt-3.5-turbo"
    model = "gpt-4"
    output_widget.clear_output()  # Clear previous output
    if btn.description == 'reply to this text':
        mode = "reply"
#         print("פוסט התגובה שלך מוכן:")
        x = "פוסט התגובה שלך מוכן:"
    elif btn.description == 'my style':
        mode = "mystyle"
        x = "הנה הפוסט שיצרנו בשבילך, על סמך הסגנון שלך"
    elif btn.description == 'rewrite':
        mode = "rewrite"
        x= "הנה הגרסא המשוכתבת של הפוסט שלך"
    elif btn.description == 'generate a new post':
        mode = "completely_new"
        text = "completely_new"
    output = generate_textual_response(text, mode,temperture_input,max_tokens_input, model)
    translation_text = output
    date_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    new_row = pd.DataFrame({'Mode': [mode], 'Text': [text], 'Response': [output], 'Date': [date_time]})
    responses_df = pd.concat([responses_df, new_row], ignore_index=True)
    clear_and_display_output(x + '\n' + output)



btn_rep.on_click(on_btn_click)
btn_my_style.on_click(on_btn_click)
btn_rewrite.on_click(on_btn_click)
btn_new.on_click(on_btn_click)

In [62]:
from IPython.display import display, HTML

# Custom CSS styles for the headline
headline_style = {
    'background-color': 'lightblue',
    'color': 'navy',
    'padding': '10px',
    'font-weight': 'bold',
    'font-size': '24px',
    'border-radius': '5px'
}

In [63]:
center_alignment = {'justify_content': 'center', 'align_items': 'center'}

# Create Image widget
image_path = 'logo.jpg'
image_widget = widgets.Image(value=open(image_path, 'rb').read(), format='png', layout=center_alignment)

# Create HBox to center image_widget
image_box = widgets.HBox([image_widget], layout=center_alignment)

# Display the image
display(image_box)

# Set image dimensions
image_widget.layout.width = '200px'
image_widget.layout.height = '200px'

# Create headline widget
headline_widget = widgets.Label(value='Let us help you make your text better! Enter input and click on the desired mode', 
                                 style=headline_style, layout=center_alignment)

# Display the headline widget
display(headline_widget)

# Create buttons HBox
buttons = widgets.HBox([btn_rep, btn_my_style, btn_rewrite, btn_new], layout=center_alignment)
display(buttons)

# Create HBox for text input
text_input_box = widgets.HBox([text_input], layout=center_alignment)
text_input.observe(on_text_change, names='value')
display(text_input_box)

# Display sliders with custom labels
length_slider_labels = widgets.HBox([widgets.Label(value='shorter!', layout=widgets.Layout(width='70px', text_align='right')),
                                     length_slider,
                                     widgets.Label(value='longer!', layout=widgets.Layout(width='70px', text_align='left'))], layout=center_alignment)

creativity_slider_labels = widgets.HBox([widgets.Label(value='less creative!', layout=widgets.Layout(width='90px', text_align='right')),
                                         creativity_slider,
                                         widgets.Label(value='more creative!', layout=widgets.Layout(width='90px', text_align='left'))], layout=center_alignment)

display(length_slider_labels)
display(creativity_slider_labels)

# Display output widget
display(output_widget)

# Display translate and regenerate buttons
trn_regen_btns = widgets.HBox([btn_translate, btn_regen], layout=center_alignment)
display(trn_regen_btns)

# Display translation output
display(translation_output)

Label(value='Let us help you make your text better! Enter input and click on the desired mode', layout=Layout(…

Output()

Output()

In [64]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
def display_response(index):
    with output:
        clear_output(wait=True)
        display(widgets.HTML(f"{index+1}/{len(responses_df)}: {responses_df.loc[index, 'Response']}"))
    with date_output:
        clear_output(wait=True)
        display(widgets.HTML(f"<div style='text-align: center; font-size: 14px; font-weight: bold;'>Date: {responses_df.loc[index, 'Date']}</div>"))

# Define arrow click functions
def on_prev_arrow_clicked(b):
    global index
    index = (index - 1) % len(responses_df)
    display_response(index)

def on_next_arrow_clicked(b):
    global index
    index = (index + 1) % len(responses_df)
    display_response(index)



In [65]:
if(responses_df.size !=0):
    
    # Create initial display
    output = widgets.Output(layout={'border': '1px solid black', 'padding': '5px', 'background-color': '#f9f9f9'})
    date_output = widgets.Output()
    # Initialize index and display first response
    index = 0
    display_response(index)
    # Create arrows
    prev_arrow = widgets.Button(description='⬅', button_style='info', layout={'width': '50px', 'height': '50px'})
    next_arrow = widgets.Button(description='➡', button_style='info', layout={'width': '50px', 'height': '50px'})

    # Link arrow clicks to functions
    prev_arrow.on_click(on_prev_arrow_clicked)
    next_arrow.on_click(on_next_arrow_clicked)

    # Display arrows on the sides of the output
    arrows_box = widgets.HBox([prev_arrow, output, next_arrow], layout={'justify_content': 'center'})
    display(widgets.VBox([arrows_box, date_output], layout={'align_items': 'center', 'justify_content': 'center'}))